In [9]:
import argparse
import os
os.environ['SPARK_VERSION'] = '3.1'
os.environ['JAVA_HOME'] = '/Library/Java/JavaVirtualMachines/jdk-11.0.12.jdk/Contents/Home'

In [5]:
import pydeequ
from pydeequ.analyzers import *
from pyspark.sql import SparkSession

In [10]:
spark = SparkSession\
    .builder\
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)\
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)\
    .getOrCreate()

22/10/10 23:22:58 WARN Utils: Your hostname, MAC1235OKKO.local resolves to a loopback address: 127.0.0.1; using 192.168.100.5 instead (on interface en0)
22/10/10 23:22:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/y.chernigin/Library/Python/3.8/lib/python/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/y.chernigin/.ivy2/cache
The jars for the packages stored in: /Users/y.chernigin/.ivy2/jars
com.amazon.deequ#deequ added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c62ffbcd-98a3-43e4-92b4-cd286a95edf9;1.0
	confs: [default]
	found com.amazon.deequ#deequ;1.2.2-spark-3.0 in central
	found org.scalanlp#breeze_2.12;0.13.2 in central
	found org.scalanlp#breeze-macros_2.12;0.13.2 in central
	found org.scala-lang#scala-reflect;2.12.1 in central
	found com.github.fommil.netlib#core;1.1.2 in central
	found net.sf.opencsv#opencsv;2.3 in central
	found com.github.rwl#jtransforms;2.4.0 in central
	found junit#junit;4.8.2 in central
	found org.apache.commons#commons-math3;3.2 in central
	found org.spire-math#spire_2.12;0.13.0 in central
	found org.spire-math#spire-macros_2.12;0.13.0 in central
	found org.typelevel#machinist_2.12;0.6.1 in central
	found com.chuusai#shapeless_2.12;2.3.2 in central
	found org.typelevel#macro-compat_

In [11]:
df = spark.read.parquet('./data/data.parquet')

In [13]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- vendor_code: string (nullable = true)
 |-- name: string (nullable = true)
 |-- type: string (nullable = true)
 |-- label: string (nullable = true)
 |-- price: double (nullable = true)
 |-- discount: double (nullable = true)
 |-- available_count: integer (nullable = true)
 |-- preorder_count: integer (nullable = true)



## PyDeeque Analyzer Research

In [14]:
analyzer = AnalysisRunner(spark)\
    .onData(df)\
    .addAnalyzer(Size())\
    .addAnalyzer(Completeness('name'))\
    .run()

In [16]:
success_metrics_df = AnalyzerContext.successMetricsAsDataFrame(spark, analyzer)

In [17]:
success_metrics_df.show()

+-------+--------+------------+--------+
| entity|instance|        name|   value|
+-------+--------+------------+--------+
|Dataset|       *|        Size|100000.0|
| Column|    name|Completeness|     1.0|
+-------+--------+------------+--------+

